# Consolidating several mappings

In [1]:
import pandas as pd
import numpy as np
import bw2data as bd
from mescal import *

In [2]:
bd.projects.set_current('ei3.8-mescal')

In [3]:
tech_CH = pd.read_csv('energyscope_data/QC/hidden/tech_CH.csv') # mapping from ecoinvent 3.8 for CH
tech_QC = pd.read_csv('energyscope_data/QC/hidden/tech_QC.csv') # mapping from ecoinvent 3.8 and premise specific for QC
comp_CH = pd.read_excel('energyscope_data/QC/hidden/techno_compositions_CH.xlsx') # list of compositions of technologies with premise mapping for CH
comp_QC = pd.read_excel('energyscope_data/QC/hidden/techno_compositions_QC.xlsx') # list of compositions of technologies with premise mapping for QC
dict_ES = pd.read_csv('energyscope_data/QC/hidden/Technology_Dictionary_v2.csv')
region_tech_ES = pd.read_excel('energyscope_data/QC/hidden/Technologies_ES_version.xlsx')

In [4]:
if 'Validation' in tech_CH.columns:
    tech_CH.drop(columns='Validation', inplace=True)
if 'Validation' in tech_QC.columns:
    tech_QC.drop(columns='Validation', inplace=True)

## Mapping file

In [5]:
len(tech_CH.ES_name.unique())

266

### QC

We start from the consolidated file of CH and add/replace what is in the tech_QC additional mapping, and filter what was only for CH using the list of technologies from ES-QC.

In [6]:
region_tech_ES.dropna(subset=['ES_version'], inplace=True) # OTHER_BIOMASS to remove

In [7]:
list_tech_QC = list(region_tech_ES[region_tech_ES.ES_version.str.contains('CA')].tech_name)

In [8]:
sub_comp_CH = list(set([x for xs in comp_CH.iloc[:, 1:].values.tolist() for x in xs])) # list of all subcomponents for CH
sub_comp_QC = list(set([x for xs in comp_QC.iloc[:, 1:].values.tolist() for x in xs])) # list of all subcomponents for QC

In [9]:
# Remove technologies that are not in ES-QC
tech_not_QC = []

# Operation
for tech in list(tech_CH[tech_CH.type == 'Operation'].ES_name):
    if tech not in list_tech_QC:
        tech_not_QC.append(tech)
    else:
        pass

# Construction
for tech in list(tech_CH[tech_CH.type == 'Construction'].ES_name):
    if tech in sub_comp_CH:
        if tech not in sub_comp_QC:
            tech_not_QC.append(tech)
        else:
            pass

In [10]:
list(set(tech_not_QC))

['BUS_FC_HYBRID_CH4',
 'CAR_ETOH_E85_LONGD',
 'CAR_ETOH_E10_LOCAL',
 'WIND',
 'CAR_GASOLINE_LONGD',
 'TRAIN_ELEC',
 'CAR_DIESEL_LONGD',
 'BUS_CNG_STOICH',
 'CAR_PHEV_LOCAL',
 'TRAIN_FREIGHT_WAG',
 'CAR_MEOH_LOCAL',
 'COACH_HY_DIESEL',
 'CAR_ETOH_E10_LONGD',
 'CAR_HEV_LOCAL',
 'COMMUTER_RAIL_DIESEL',
 'CAR_FC_CH4_LOCAL',
 'CAR_DIESEL_LOCAL',
 'CAR_NG_LOCAL',
 'TRUCK_EV',
 'CAR_GASOLINE_LOCAL',
 'TRUCK_FC',
 'CAR_BEV_LOWRANGE',
 'CAR_BEV_MEDRANGE_LONGD',
 'CAR_DME_D10_LONGD',
 'COACH_FC_HYBRID_H2',
 'TRAIN_FREIGHT_NG_WAG',
 'BUS_FC_HYBRID_H2',
 'CAR_PHEV_LONGD',
 'COMMUTER_RAIL_ELEC',
 'CAR_HEV_LONGD',
 'COACH_FC_HYBRID_CH4',
 'TRAIN_FREIGHT_LOC',
 'TRUCK_SNG',
 'CAR_FC_CH4_LONGD',
 'CAR_NG_LONGD',
 'TRAIN_FREIGHT',
 'COACH_CNG_STOICH',
 'CAR_BEV_MEDRANGE_LOCAL',
 'TRUCK',
 'TRAIN_NG',
 'CAR_DME_D10_LOCAL',
 'CAR_FC_H2_LOCAL',
 'CAR_FC_H2_LONGD',
 'TRAIN_FREIGHT_NG',
 'TRAIN_FREIGHT_NG_LOC',
 'CAR_MEOH_LONGD',
 'CAR_ETOH_E85_LOCAL']

In [11]:
tech_CH_filtered = tech_CH.drop(index=tech_CH[tech_CH.ES_name.isin(tech_not_QC)].index)

In [12]:
# Remove the LCI datasets that need to be updated from the CH list
update_constr = []
for tech in list(tech_CH[tech_CH.type == 'Construction'].ES_name):
    if (tech in list(tech_CH[tech_CH.type == 'Construction'].ES_name)) & (tech in list(tech_QC[tech_QC.type == 'Construction'].ES_name)):
        update_constr.append(tech)

update_op = []
for tech in list(tech_CH[tech_CH.type == 'Operation'].ES_name):
    if (tech in list(tech_CH[tech_CH.type == 'Operation'].ES_name)) & (tech in list(tech_QC[tech_QC.type == 'Operation'].ES_name)):
        update_op.append(tech)

In [13]:
tech_CH_filtered.drop(index=tech_CH_filtered[(tech_CH_filtered.ES_name.isin(update_constr)) & (tech_CH_filtered.type == 'Construction')].index, inplace=True)
tech_CH_filtered.drop(index=tech_CH_filtered[(tech_CH_filtered.ES_name.isin(update_op)) & (tech_CH_filtered.type == 'Operation')].index, inplace=True)

In [14]:
tech_consolidated_QC = pd.concat([tech_CH_filtered, tech_QC])

In [15]:
tech_consolidated_QC.duplicated(subset=['ES_name', 'type']).sum()

0

## Unit conversion file

In [16]:
# # Allows to keep formulas in Excel files
# from openpyxl import load_workbook
# wb_CH = load_workbook(filename = 'energyscope_data/hidden/tech_unit_conversion_CH.xlsx')
# unit_conv_CH = pd.DataFrame(wb_CH[wb_CH.sheetnames[0]].values)
# wb_QC = load_workbook(filename = 'energyscope_data/hidden/tech_unit_conversion_QC.xlsx')
# unit_conv_QC = pd.DataFrame(wb_QC[wb_QC.sheetnames[0]].values)

In [17]:
# # setting first row as header
# new_header_CH = unit_conv_CH.iloc[0]
# unit_conv_CH = unit_conv_CH[1:]
# unit_conv_CH.columns = new_header_CH
# new_header_QC = unit_conv_QC.iloc[0]
# unit_conv_QC = unit_conv_QC[1:]
# unit_conv_QC.columns = new_header_QC

In [18]:
unit_conv_CH = pd.read_excel('energyscope_data/QC/hidden/tech_unit_conversion_CH.xlsx')
unit_conv_QC = pd.read_excel('energyscope_data/QC/hidden/tech_unit_conversion_QC.xlsx')

### CH

In [19]:
unit_conv_CH = unit_conv_CH[['ES_name', 'ei_constr_unit', 'ES_constr_unit', 'ei_use_unit', 'ES_use_unit', 'capacity', 'conversion', 'ei_constr_unit_size', 'ES_constr_unit_size', 'Assumptions & Sources']]

In [20]:
# Drop the rows where both the capacity and conversion factors are None
unit_conv_CH.drop(unit_conv_CH[(unit_conv_CH.conversion.values == None) & (unit_conv_CH.capacity.values == None)].index, inplace=True)

### QC

In [21]:
unit_conv_QC = unit_conv_QC[unit_conv_CH.columns]

In [22]:
unit_conv_QC.dropna(how='all', axis=0, inplace=True)

In [23]:
# In order to overwrite some conversion factors (same technologies but different factors between CH and QC), we remove from the CH file the factors that are present in both files
unit_conv_CH_overwrite = unit_conv_CH.copy()
for tech in list(unit_conv_QC.ES_name.unique()):
    if tech in list(unit_conv_CH.ES_name.unique()):
        unit_conv_CH_overwrite.drop(unit_conv_CH[unit_conv_CH.ES_name == tech].index, inplace=True)

In [24]:
unit_conv_QC_consolidated = pd.concat([unit_conv_CH_overwrite.drop(unit_conv_CH_overwrite[unit_conv_CH_overwrite.ES_name.isin(tech_not_QC)].index), unit_conv_QC])

# Model file

In [25]:
ES_region = 'CH'

In [26]:
layers_in_out = pd.read_csv(f"energyscope_data/QC/hidden/layers_in_out_{ES_region}.csv")

In [27]:
if ES_region == 'QC':
    model = layers_in_out.melt(id_vars=['ES_name'], value_vars=layers_in_out.columns[1:])
    model = model[model['value'] != 0]
    model.rename(columns={'ES_name': 'Flow', 'variable': 'Name', 'value': 'Amount'}, inplace=True)
elif ES_region == 'CH':
    model = layers_in_out
else:
    raise ValueError('ES_region should be either CH or QC')
model[['Name', 'Flow', 'Amount']].to_csv(f'energyscope_data/{ES_region}/model.csv', index=False)

# Duplicate mapping for mobility models

In [28]:
if ES_region == 'QC':
    tech_ecoinvent = tech_consolidated_QC.copy(deep=True)
    tech_unit_conversion = unit_conv_QC_consolidated.copy(deep=True)
    comp = comp_QC.copy(deep=True)
elif ES_region == 'CH':
    tech_ecoinvent = tech_CH.copy(deep=True)
    tech_unit_conversion = unit_conv_CH_overwrite.copy(deep=True)
    comp = comp_CH.copy(deep=True)
else:
    raise ValueError('ES_region should be either CH or QC')

In [29]:
tech_ecoinvent.reset_index(drop=True, inplace=True)
tech_unit_conversion.reset_index(drop=True, inplace=True)
comp.reset_index(drop=True, inplace=True)

In [31]:
assumptions_diff = pd.read_excel(f'energyscope_data/QC/hidden/assumptions_diff_{ES_region}.xlsx')
mob_model_private = pd.read_csv(
    f"energyscope_data/QC/hidden/MODELS_OF_TECHNOLOGIES_OF_PRIVATEMOB_ALL_DISTANCES_{ES_region}.csv", sep=',')
if ES_region == 'QC':
    mob_model_public = pd.read_csv(
        f"energyscope_data/QC/hidden/MODELS_OF_TECHNOLOGIES_OF_PUBLICMOB_ALL_DISTANCES_{ES_region}.csv", sep=',')
    mob_model_freight = pd.read_csv(
        f"energyscope_data/QC/hidden/MODELS_OF_TECHNOLOGIES_OF_FREIGHTMOB_ALL_DISTANCES_{ES_region}.csv", sep=',')

In [32]:
# Gather all non-nan components into a list
comp['Components'] = [[e for e in row if e == e] for row in comp.iloc[:, 1:].values.tolist()]
comp_dict = dict(zip(comp.ES_name, comp.Components))
N_sub_comp_max = 4  # maximum number of subcomponents in the compositions file

In [33]:
def gen_df_mob_models(df):
    df_mobility_models = pd.DataFrame(columns=tech_ecoinvent.columns)

    for i in range(len(df)):
        tech = df.Main_tech.iloc[i]

        if tech in list(tech_ecoinvent.ES_name):

            j = 1
            model = str(df[df.Main_tech == tech][f'Model_{j}'].iloc[0])
            while (model != 'nan') & (j < df.shape[1]):
                if str(df_mobility_models.index.max()) == 'nan':
                    idx = 1
                else:
                    idx = df_mobility_models.index.max() + 1
                df_mobility_models.loc[idx] = [model] + list(tech_ecoinvent[tech_ecoinvent.ES_name == tech].iloc[0, 1:])  # operation
                tech_unit_conversion.loc[tech_unit_conversion.index.max() + 1] = [model] + list(tech_unit_conversion[tech_unit_conversion.ES_name == tech].iloc[0,1:])  # update unit conversion Excel files with additional rows for mobility models
                dict_ES.loc[dict_ES.index.max() + 1] = [model] + list(dict_ES[dict_ES['Programming name'] == tech].iloc[0,1:])  # update technology dictionary Excel file with additional rows for mobility models
                assumptions_diff.loc[assumptions_diff.index.max() + 1] = [model] + list(assumptions_diff[assumptions_diff.ES_name == tech].iloc[0,1:])  # update unit conversion Excel files with additional rows for mobility models

                if tech in comp_dict.keys():

                    N_sub_comp = len(comp_dict[tech])
                    subscript_comp_list = []

                    for i, sub_comp in enumerate(comp_dict[tech]):
                        subscript_comp = sub_comp.replace(tech, '')
                        subscript_comp_list.append(subscript_comp)
                        df_mobility_models.loc[df_mobility_models.index.max() + 1] = [model + subscript_comp] + list(tech_ecoinvent[tech_ecoinvent.ES_name == sub_comp].iloc[0,1:])  # construction component idx
                        tech_unit_conversion.loc[tech_unit_conversion.index.max() + 1] = [model + subscript_comp] + list(tech_unit_conversion[tech_unit_conversion.ES_name == sub_comp].iloc[0,1:])  # update unit conversion Excel files
                        assumptions_diff.loc[assumptions_diff.index.max() + 1] = [model + subscript_comp] + list(assumptions_diff[assumptions_diff.ES_name == sub_comp].iloc[0,1:])  # update unit conversion Excel files

                    comp.loc[comp.index.max() + 1] = [model] + [model + a for a in subscript_comp_list] + [np.nan] * (N_sub_comp_max - N_sub_comp) + [[model + a for a in subscript_comp_list]]  # update the compositions Excel files
                    comp_dict[model] = [model + a for a in subscript_comp_list]

                else:
                    df_mobility_models.loc[idx + 1] = [model] + list(
                        tech_ecoinvent[tech_ecoinvent.ES_name == tech].iloc[1, 1:])  # construction
                j += 1
                if j < df.shape[1]:
                    model = str(df[df.Main_tech == tech][f'Model_{j}'].iloc[0])

    return df_mobility_models

In [34]:
if ES_region == 'QC':
    basic_tech_to_remove = list(mob_model_private.Main_tech) + list(mob_model_public.Main_tech) + list(
        mob_model_freight.Main_tech)
else:
    basic_tech_to_remove = list(mob_model_private.Main_tech)

for tech in basic_tech_to_remove:
    if tech in comp_dict.keys():  # add the subcomponents to the list of technologies to remove
        for sub_comp in comp_dict[tech]:
            basic_tech_to_remove.append(sub_comp)

In [35]:
# Create df of mapping with mobility models
df_mobility_models_private = gen_df_mob_models(mob_model_private)
if ES_region == 'QC':
    df_mobility_models_public = gen_df_mob_models(mob_model_public)
    df_mobility_models_freight = gen_df_mob_models(mob_model_freight)

# Remove the mobility basic technologies
tech_ecoinvent.drop(tech_ecoinvent[tech_ecoinvent.ES_name.isin(basic_tech_to_remove)].index, inplace=True)
tech_unit_conversion.drop(tech_unit_conversion[tech_unit_conversion.ES_name.isin(basic_tech_to_remove)].index,
                          inplace=True)
comp.drop(comp[comp.ES_name.isin(basic_tech_to_remove)].index, inplace=True)
assumptions_diff.drop(assumptions_diff[assumptions_diff.ES_name.isin(basic_tech_to_remove)].index, inplace=True)

In [36]:
mob_model_comp = []  # list of components for mobility technologies composition (to remove)

if ES_region == 'CH':
    mob_tech_list = list(mob_model_private.Main_tech)
else:
    mob_tech_list = list(mob_model_private.Main_tech) + list(mob_model_public.Main_tech) + list(
        mob_model_freight.Main_tech)

for mob_tech in mob_tech_list:
    if mob_tech in comp_dict.keys():
        for sub_comp in comp_dict[mob_tech]:
            mob_model_comp.append(sub_comp)
    else:
        pass

tech_ecoinvent.drop(tech_ecoinvent[tech_ecoinvent.ES_name.isin(mob_model_comp)].index, inplace=True)

In [37]:
# Concatenate the overall df's
if ES_region == 'CH':
    tech_ecoinvent = pd.concat([tech_ecoinvent,
                                df_mobility_models_private])
else:
    tech_ecoinvent = pd.concat([tech_ecoinvent,
                                df_mobility_models_private,
                                df_mobility_models_public,
                                df_mobility_models_freight])
tech_ecoinvent = tech_ecoinvent.sort_values('ES_name').reset_index(drop=True)

# Mapping file with both technologies and resources

In [38]:
res = pd.read_csv(f"energyscope_data/QC/hidden/res_ecoinvent.csv")
flows = pd.read_csv('energyscope_data/QC/hidden/flows_ecoinvent.csv') 

In [39]:
db_flows = concatenate_databases(list(flows.Database.unique()))

In [40]:
flows['Type'] = len(flows) * ['Flow']

In [41]:
# Define the user-defined ranking
if ES_region == 'QC':
    my_ranking = [
        'CA-QC',  # Quebec
        'CA',  # Canada
        'CA-ON',  # Other canadian provinces 
        'CA-AB',
        'CA-BC',
        'CA-MB',
        'CA-NB',
        'CA-NF',
        'CA-NS',
        'CA-NT',
        'CA-NU',
        'CA-PE',
        'CAZ',  # Canada - Australia - New Zealand
        'RNA',  # North America
        'US',  # United States
        'USA',  # United States
        'GLO',  # Global average 
        'RoW',  # Rest of the world
    ]
elif ES_region == 'CH':
    my_ranking = [
        'CH', 
        'RER', 
        'IAI Area, EU27 & EFTA',
        'NEU',
        'EUR',
        'GLO',
        'RoW'
    ]
else:
    raise ValueError('ES_region should be either CH or QC')

In [42]:
flows = change_location_mapping_file(
    flows,
    my_ranking,
    db_flows,
    ES_region,
)

In [43]:
res.drop(columns=['Description'], inplace=True)
res.dropna(subset=['product_name'], inplace=True)
res['type'] = len(res) * ['Resource']
mapping = pd.concat([tech_ecoinvent, res], ignore_index=True).rename(
    columns={'ES_name': 'Name', 'type': 'Type', 'product_name': 'Product', 'activity_name': 'Activity', 'region': 'Location', 'unit': 'Unit', 'database': 'Database'})
mapping = pd.concat([mapping, flows])

## Adapting trucks to carculator names

In [44]:
def change_truck_name(row):
    if row.Name.startswith('LCV_') | row.Name.startswith('SEMI_SH_'):
        if row.Type == 'Operation':
            row.Product = row.Product.replace('freight, lorry', 'truck')
            row.Activity = row.Activity.replace('freight, lorry', 'truck')
        
        elif row.Type == 'Construction':
            row.Product = row.Product.replace('Light duty ', '')
            row.Activity = row.Activity.replace('Light duty ', '')
            
            row.Product = row.Product.replace('Medium duty ', '')
            row.Activity = row.Activity.replace('Medium duty ', '')
        
        row.Product = row.Product.replace(' gross weight', '')
        row.Activity = row.Activity.replace(' gross weight', '')
        
        row.Product = row.Product.replace('EURO-VI', 'Euro-6')
        row.Activity = row.Activity.replace('EURO-VI', 'Euro-6')
        
        row.Location = row.Location.replace('RER', 'CH')
        
        if 'urban delivery' in row.Activity:
            row.Database = row.Database.replace('lci-long_haul_trucks', 'urban delivery_truck')
        elif 'regional delivery' in row.Activity:
            row.Database = row.Database.replace('lci-long_haul_trucks', 'regional delivery_truck')
        else:
            raise ValueError('Truck type not recognized')
        
    return row

In [45]:
mapping = mapping.apply(change_truck_name, axis=1)

In [46]:
mapping.to_csv(f"energyscope_data/{ES_region}/mapping.csv", index=False)

# Composition file

In [47]:
comp.rename(columns={'ES_name': 'Name'}, inplace=True)
comp[['Name', 'Components']].to_csv(f"energyscope_data/{ES_region}/technology_compositions.csv", index=False)

# Unit conversion and assumptions files

In [48]:
res_unit_conversion = pd.read_excel("energyscope_data/QC/hidden/res_unit_conversion.xlsx")
other_unit_conversion = pd.read_csv("energyscope_data/QC/hidden/other_unit_conversion.csv")
lifetime = assumptions_diff.copy(deep=True)

In [49]:
tech_unit_conversion_melted = tech_unit_conversion[['ES_name', 'capacity', 'conversion', 'ei_constr_unit', 'ES_constr_unit', 'ei_use_unit', 'ES_use_unit']].rename(
    columns={'ES_name': 'Name', 'capacity': 'Construction', 'conversion': 'Operation'}
).melt(
    id_vars='Name',
    value_vars=['Construction', 'Operation'],
    var_name='Type',
    value_name='Value'
).sort_values('Name').dropna(subset='Value')

In [50]:
tech_unit_conversion_melted_constr = tech_unit_conversion_melted[tech_unit_conversion_melted.Type == 'Construction']
tech_unit_conversion_melted_op = tech_unit_conversion_melted[tech_unit_conversion_melted.Type == 'Operation']
tech_unit_conversion_melted_constr = tech_unit_conversion_melted_constr.merge(tech_unit_conversion[['ES_name', 'ei_constr_unit', 'ES_constr_unit']], left_on='Name', right_on='ES_name').rename(columns={'ei_constr_unit': 'To', 'ES_constr_unit': 'From'}).drop(columns='ES_name')
tech_unit_conversion_melted_op = tech_unit_conversion_melted_op.merge(tech_unit_conversion[['ES_name', 'ei_use_unit', 'ES_use_unit']], left_on='Name', right_on='ES_name').rename(columns={'ei_use_unit': 'To', 'ES_use_unit': 'From'}).drop(columns='ES_name')
tech_unit_conversion_melted = pd.concat([tech_unit_conversion_melted_constr, tech_unit_conversion_melted_op], ignore_index=True).sort_values('Name')

In [51]:
res_unit_conversion_melted = res_unit_conversion[['ES_name', 'conversion', 'ei_unit', 'ES_unit']].rename(
    columns={'ES_name': 'Name', 'conversion': 'Resource'}
).melt(
    id_vars='Name', 
    value_vars=['Resource'],
    var_name='Type', 
    value_name='Value'
).sort_values('Name').dropna(subset='Value')

In [52]:
res_unit_conversion_melted = res_unit_conversion_melted.merge(res_unit_conversion[['ES_name', 'ei_unit', 'ES_unit']], left_on='Name', right_on='ES_name').rename(columns={'ei_unit': 'To', 'ES_unit': 'From'}).drop(columns='ES_name')

In [53]:
unit_conversion = pd.concat([tech_unit_conversion_melted, 
                             res_unit_conversion_melted, 
                             other_unit_conversion[['Name', 'Value', 'Type', 'From', 'To']],
                             ], ignore_index=True).sort_values('Name')

In [54]:
unit_conversion.to_csv(f"energyscope_data/{ES_region}/unit_conversion.csv", index=False)

In [55]:
lifetime[['ES_name', 'lifetime_ES', 'lifetime_ei']].rename(columns={'ES_name': 'Name', 'lifetime_ES': 'ESM', 'lifetime_ei': 'LCA'}).to_csv(f"energyscope_data/{ES_region}/lifetime.csv", index=False)